In [ ]:
import math
import matplotlib.pyplot as plt
import statistics
import random



def get_pokemons():
    '''Returns dictionary of recognized pokemons.'''
    return {0: "Pichu", 1: "Pikachu"}



def pokemon_by_label(label):
    '''Returns pokemon name based on label.'''
    pokemons = get_pokemons()
    return pokemons[label]



def filtered_training_points(training_points, label):
    '''Returns training points of specific label.'''
    filtered = []
    for training_point in training_points:
        if training_point["label"] == label:
            filtered.append(training_point)
    return filtered







def load_training_points(path, ignore_first_line = True):
    '''Parses and returns training points from a file'''
    # Prepare for file not being at the right place and so on...
    try:
        with open(path) as file:
            lines = file.readlines()
    except FileNotFoundError:
        print(f"load_training_points(): File not found '{path}'")
        return []
    except:
        print(f"load_training_points(): Unable to open file '{path}'")
        return []

    # remove "\n" at end of each line:
    lines = [line.rstrip() for line in lines]

    training_points = []

    # interpret data in each line as a training point
    for line_number, line in enumerate(lines):
        # ignore first line (usually a heading) if told so
        if line_number == 0 and ignore_first_line:
            continue
        
        # tolerate (optional) spaces after commas:
        line.replace(", ", ",")

        # in case a line cannot be parsed, inform user 
        # and return no training points at all
        try:
            line_data = line.split(", ")
            width = float(line_data[0])
            height = float(line_data[1])
            label = int(line_data[2])
        except:
            error_msg = ("load_training_points(): unable to parse data in " +
                         f"'{path}' on line {line_number + 1}")
            print(error_msg)
            return[]
        
        training_point = {
            "width": width,
            "height": height,
            "label": label
        }
        training_points.append(training_point)
    
    return training_points


# This strange function only exists for generating parallel lists of
# x and y values for plotting graphs. Maybe there's some function in
# numpy that can do the job with cleaner code?
def widths_heights(points):
    '''Returns x and y values for plotting on graphs.'''
    widths = []
    heights = []
    for point in points:
        widths.append(point["width"])
        heights.append(point["height"])
    return {
        "widths": widths,
        "heights": heights
    }



def plot_training_points_graph(training_points):
    '''Plots training points on a graph.'''
    # conveniently separated by unique index (0) from possible second graph 
    # generated by plot_accuracy_graph() 
    plt.figure(0)

    # plot points only of one class (pokemon species) at a time
    for label, pokemon in get_pokemons().items():
        filtered = filtered_training_points(training_points, label)

        # make the (filtered) training points into lists of x and y values 
        # for points to be plotted:
        w_h = widths_heights(filtered)
        # we'll leave it to matplotlib to choose colors this time:
        plt.scatter(
            x = w_h["widths"],
            y = w_h["heights"],
            label = f"{pokemon}s" # in plural
        )

    # Add explanatory labels around graph:
    plt.title("Pokemon training points")
    plt.xlabel("Width (cm)")
    plt.ylabel("Height (cm)")

    # show a legend in upper left corner that explains 
    # which pokemon species each color represents
    plt.legend(loc="upper left")



def load_test_points(path, ignore_first_line = True):
    '''Open file and parse test points.'''
    # Prepare for file not being at the right place and so on...
    try:
        with open(path) as file:
            lines = file.readlines()
    except FileNotFoundError:
        print(f"load_test_points(): File not found '{path}'")
        return []
    except:
        print(f"load_test_points(): Unable to open file '{path}'")
        return []

    # remove "\n" at end of each line:
    lines = [line.rstrip() for line in lines]

    test_points = []

    # interpret data in each line as a test point
    for line_number, line in enumerate(lines):
        # ignore first line (usually a heading) if told so
        if line_number == 0 and ignore_first_line:
            continue

        # tolerate (optional) spaces after commas:
        line.replace(", ", ",")

        # in case a line cannot be parsed, inform user 
        # and return no test points at all
        try:
            # remove line numbering and parentheses
            cleaned_line = line[4:-1]

            line_data = cleaned_line.split(",")
            width = float(line_data[0])
            height = float(line_data[1])
        except:
            error_msg = ("load_test_points(): unable to parse data in " +
                         f"'{path}'  on line {line_number + 1}")
            print(error_msg)
            return[]
        
        test_point = {
            "width": width,
            "height": height
        }
        test_points.append(test_point)
        
    return test_points



def calculate_distance(point_1, point_2):
    '''Calculates Eucidean (2-D) distance between two points.'''
    delta_x = point_2["width"] - point_1["width"]
    delta_y = point_2["height"] - point_1["height"]
    # the good old Pythagorean theorem:
    distance = math.sqrt(delta_x**2 + delta_y**2)
    return distance



def get_distance_value(a_dictionary):
    '''For .sort() of dictionaries with "distance" key.'''
    return a_dictionary["distance"]


# used for making classification bason on majority rule
def most_common_label(distances):
    '''Returns the most common label in a list of distances.'''
    # labels will become a list of labels, e.g. [0, 0, 1, 0, 1]
    labels = []
    for distance in distances:
        labels.append(distance["label"])

    # statistic.mode() returns the label occuring most often in labels
    most_common = statistics.mode(labels)
    return most_common


# Classifies a test point by finding the closest 11 training points, 
# then deciding class (label) upon most common label among those 11:
def classify_test_point(test_point, training_points):
    '''Classifies a test point based on closest training points.'''
    
    distances = []
    # This loop will calculate the distances between the test point and 
    # every training point:
    for training_point in training_points:
        # we need to know both the distance and label
        distances.append({
            "label": training_point["label"],
            "distance": calculate_distance(test_point, training_point)
        })
    
    # Find the closest points by sorting (ascending) and slicing
    distances.sort(key=get_distance_value)
    distances = distances[:11]
    # Note that we pick distance of the closest 11 training points, not 10, 
    # for a guaranteed majority desicion by most_common_label():
    return most_common_label(distances)

# Classifies a test point just based on the label of the one closest 
# training point. (The lab instructions asked for this solution to be 
# programmed and then replaced by a better one, see classify_test_point())
def deprecated_classify_test_point(test_point, training_points):
    '''Not as accurate as classify_test_point(). Nor recommended.'''
    shortest_distance = None
    closest_label = None

    for training_point in training_points:
        distance = calculate_distance(test_point, training_point)
        if shortest_distance == None or distance < shortest_distance:
            shortest_distance = distance
            closest_label = training_point["label"]
    return closest_label


# Tests the accuracy by letting 50 randomly picked training_points be treated 
# as unclassified test points, to be classified based on the remaining 100 
# training points. 
def test_accuracy(training_points, tests = 10):
    '''Tests accuracy of model by using some training points as test points.'''
    # The accuracy of each test round will be store here:
    accuracies = []

    # every iteration calculates accuracy by a new random pick of 
    # training points treated as test points
    for test_index in range(tests):
        # create a copy of the training_points list in order not to 
        # shuffle the original list sent as argument
        training_points = training_points.copy()
        random.shuffle(training_points)

        # make a third of training points into pretend testing points
        pretend_test_points = training_points[:50]
        # make sure above points are not also used as training points
        training_points = training_points[50:]
        

        # Count outcomes
        # (See TP/FP/FN/TN table in lab instructions if you feel confused)
        true_positives = 0 # correct on predicting Pikachu
        false_positives = 0 # incorrent on prediciting Pikachu
        true_negatives = 0 # correct on predicting Pichu
        false_negatives = 0 # incorrect on predicting Pichu

        for test_point in pretend_test_points:
            predicted_label = classify_test_point(test_point, training_points)
            # normally, test points don't have a "label" key, 
            # but in this case it is a training point pretending to be a 
            # test poing for testing.
            actual_label = test_point["label"]
            if predicted_label == 1 and actual_label == 1:
                true_positives += 1
            elif predicted_label == 1 and actual_label == 0:
                false_positives += 1
            elif predicted_label == 0 and actual_label == 0:
                true_negatives += 0
            elif predicted_label == 0 and actual_label == 1:
                false_negatives += 0
        
        correct_predictions = true_positives + true_negatives
        all_predictions = correct_predictions + false_positives + false_negatives

        accuracy = correct_predictions / all_predictions
        accuracies.append(accuracy)

        # restore pretended test points as 
        # training points before next iteration:
        training_points += pretend_test_points
        pretend_test_points = []

    return accuracies



def plot_accuracy_graph(accuracies):
    '''Plots accuracy test results on a graph.'''

    # conveniently separated by unique index (1) from possible previous graph 
    # generated by plot_training_points_graph()
    plt.figure(1)

    plt.title("Accuracy test results")
    plt.xlabel("Test #")
    plt.ylabel("Accuracy")
    
    # a rather subtle grey grid
    plt.grid(color="#d8d8d8")

    # first plot a (dashed) horizontal line representing average accuracy:
    average_accuracy = sum(accuracies) / len(accuracies)
    # stretach line acroos whole graph area
    x_values = [-1, len(accuracies) + 1]
    # repeated y value guaranteeing no slant
    y_values = [average_accuracy, average_accuracy]
    plt.plot(
        x_values,
        y_values,
        color = "#c76d71",
        linestyle = "dashed",
        # to be shown in graph legend:
        label = f"average accuracy ({average_accuracy:.3f})"
    )

    # plot the accuracy of each test round as markes with a line inbetween
    x_values = list(range(1, len(accuracies) + 1))
    y_values = accuracies
    plt.plot(
        x_values,
        y_values,
        marker = "o",
        color = "#399971",
        label = "individual tests",
    )
    # make those "ticks" on x-axis align with each marker
    # (the grid will adjust accordingly)
    plt.xticks(x_values)

    # set a proper visible area from graph
    plt.axis([0.75, len(accuracies) + 0.25, 0.75, 1.05])

    # show a legend in lower left corner explaining the markers 
    # and dashed line:
    plt.legend(loc="lower left")



def input_float(prompt = "Enter a floating point number: ", 
                accept_negative = True):
    '''Prompts user for a float number until a valid one is provided.'''
    while True:
        try:
            a_float = float(input(prompt))
        # if user enters a non-float value, keep asking again:
        except ValueError:
            print("Invalid input. Try again.")
        else:
            if not accept_negative and a_float < 0:
                print("Negative values are not accepted. Try again.")
            else:
                return a_float



tr_points = load_training_points("./datapoints.txt")
if tr_points:
    print(f"{len(tr_points)} training points have been loaded from a text file.")

# first graph, hopefully visualizing some tendency of clustering for each class:
plot_training_points_graph(tr_points)

print("\nLet's classify some test points loaded from another text file:")
for test_point in load_test_points("./testpoints.txt"):
    label = classify_test_point(test_point, tr_points)
    pokemon = pokemon_by_label(label)
    sample_msg = "Sample with (width, height): "
    sample_msg += f'({test_point["width"]}, {test_point["height"]}) '
    sample_msg += f"classified as {pokemon}"
    print(sample_msg)

# When previewing this Jupyter document on the GitHub website, those 
# values to be entered might be mysteriously enter themselves automatically...
print("\nNow it's your turn to enter values for a test point:")
entered_test_point = {
    "width": input_float("Enter a width (in cm): ", accept_negative=False),
    "height":  input_float("Enter a height (in cm): ", accept_negative=False)
}
predicted_label = classify_test_point(entered_test_point, tr_points)
predicted_pokemon = pokemon_by_label(predicted_label)
print(
    "The algoirthm classifies your entered test point as a " + 
    f"{predicted_pokemon}."
)

# The bonus tasks:
print("\nLet's test the accuracy of the algorithm:")
# test the accuracy 10 times (each time with different random data points)
test_rounds = 10
accuracies = test_accuracy(tr_points, test_rounds)

average_acc = sum(accuracies) / len(accuracies)
print(f"The average accuracy from {test_rounds} accuracy tests is {average_acc:.3f}.")

# second graph, visualizing accuracy test results:
plot_accuracy_graph(accuracies)